In [1]:
import os

import music21

score_fp = input("Input directory of musicXML score files: ")
    
if os.path.isdir(score_fp):
    score_list = [os.path.join(score_fp, path) for path in os.listdir(score_fp)]
else:
    score_list = [score_fp]

Input directory of musicXML score files: C:\Users\Danie\PycharmProjects\ChordGenerator\data\source_data\rs200_score_dt_longnotes


**bins_per_measure** identifies how many bins the script should discretize each measure of the harmony and melody into

In [2]:
bins_per_measure = 16

In [3]:
dataset_fp = input("Input directory to store training vectors: ")

if not os.path.exists(dataset_fp):
    os.mkdir(dataset_fp)

Input directory to store training vectors: C:\Users\Danie\PycharmProjects\ChordGenerator\data\training_data\training_dt_longnotes


In [4]:
for score_path in score_list:
    file_name = os.path.basename(score_path).split(".")[0]
    print("Processing:", file_name)
    
    with open(os.path.join(dataset_fp, file_name + ".txt"), "w") as f_out:
        # TODO, method of stepping through these could be more intuitive, 
         # should be based on measure content rather than derived step sizes assuming non-overstuffed measures

        # Assume 4/4 time, then total length in quarterlength is 
         #(4/4) * 4 = 4 which is divided into bins_per_measure
        step_length = 4 / bins_per_measure 
        score = music21.converter.parse(score_path)

        melody, harmony = score.parts

        melody_measures = melody.getElementsByClass(music21.stream.Measure)
        harmony_measures = harmony.getElementsByClass(music21.stream.Measure)

        for i, mel_measure in enumerate(melody_measures):
            har_measure = harmony_measures[i]
            
            if mel_measure.timeSignature is not None:
                step_length = (mel_measure.timeSignature.numerator / mel_measure.timeSignature.denominator
                               * 4 / bins_per_measure)

            melody_elements = [item for item in mel_measure.notesAndRests]
            harmony_elements = [item for item in har_measure.notesAndRests]
            melody_index = 0
            harmony_index = 0

            for i in range(bins_per_measure):
                offset_timestep = i * step_length

                if (melody_elements[melody_index] is not melody_elements[-1] 
                    and melody_elements[melody_index + 1].offset <= offset_timestep):
                    melody_index += 1
                if (harmony_elements[harmony_index] is not harmony_elements[-1] 
                    and harmony_elements[harmony_index + 1].offset <= offset_timestep):
                    harmony_index += 1

                melody_item = melody_elements[melody_index]
                harmony_item = harmony_elements[harmony_index]

                harmony_vector = ["0" for i in range(12)]
                if not harmony_item.isRest:
                    for pitch_class in harmony_item.pitchClasses:
                        harmony_vector[pitch_class] = "1"

                f_out.write(str(melody_item.pitch.pitchClass) if melody_item.name != "rest" else "12")
                f_out.write(" ")
                f_out.write("".join(harmony_vector))
                f_out.write("\n")

            f_out.write("\n")

Processing: 1999
Processing: all_along_the_watchtower
Processing: all_apologies
Processing: all_i_have_to_do_is_dream
Processing: anarchy_in_the_uk
Processing: a_change_is_gonna_come
Processing: a_day_in_the_life
Processing: a_hard_days_night
Processing: a_whiter_shade_of_pale
Processing: back_in_black
Processing: be-bop-a-lula
Processing: be_my_baby
Processing: billie_jean
Processing: bitter_sweet_symphony
Processing: bizarre_love_triangle
Processing: blitzkrieg_bop
Processing: blowin_in_the_wind
Processing: blueberry_hill
Processing: blue_suede_shoes
Processing: bohemian_rhapsody
Processing: born_to_be_wild
Processing: born_to_run
Processing: both_sides_now
Processing: bo_diddley
Processing: bridge_over_troubled_water
Processing: brown-eyed_girl
Processing: california_dreamin
Processing: california_girls
Processing: california_love
Processing: cathys_clown
Processing: changes
Processing: come_as_you_are
Processing: crazy
Processing: crying
Processing: dancing_in_the_street
Processing